<a href="https://colab.research.google.com/github/RomanGorelsky/NLP/blob/main/Variant_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## The task was done by Gorelskii Roman, student of the bachelor program "Data Science and Business Analytics". All the comments are both written in english and in russian languages.

## Задание было выполнено Горельским Романом, учеником образовательной программы бакалавриата "Прикладной анализ данных". Все комментарии написаны на английском и русском языках.

# Token classification на примере задачи NER (12 баллов)

Это домашнее задание проходит в формате peer-review. Это означает, что его будут проверять ваши однокурсники. Поэтому пишите разборчивый код, добавляйте комментарии и пишите выводы после проделанной работы.

Классификация токенов — задача, в которой для каждого отдельного токена или слова необходимо определить его тип, например, часть речи. В этом ноутбуке вам предстоит решить подвид задачи Token Classification, а именно NER или Named Entity Recognition. Вам необходимо для каждого слова определить, обозначает ли оно именованную сущность, например, имя человека, название места и тд.

Установим необходимые библиотеки: ```datasets```, ```transformers``` и ```seqeval```.

In [1]:
!pip install datasets
!pip install transformers
!pip install seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 61.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 131.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 95.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=2e6303ba8f9fa5a011ced474ea9a7402a5cb83ab9a73b7bbd1c8d595e9ccb74d
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed

In [ ]:
import pandas as pd
from tqdm.auto import tqdm

import torch
import torch.nn as nn
import numpy as np

import transformers
from transformers import (
    AutoTokenizer,
    AutoModelForTokenClassification,
    TrainingArguments,
    Trainer,
    DataCollatorForTokenClassification
    )
from datasets import load_dataset, load_metric

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cpu'

## Подготовка данных

Давайте поближе познакомимся с тем, как хранятся датасеты для NER. В этом задании вам предстоит работать с conll2003. Подробнее о нем можно узнать по этой [ссылке](https://huggingface.co/datasets/conll2003).

В качестве предобученной модели воспользуемся DistilBERT. Это уменьшенная версия обычного BERT.

In [ ]:
model_checkpoint = "distilbert-base-uncased"
batch_size = 64

Загрузим данные с помощью функции load_dataset.

In [ ]:
datasets = load_dataset("conll2003")

Наши данные состоят из следующих выборок:

In [ ]:
datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3453
    })
})

В NER существует сразу несколько типов лэйблов для каждого токена. В случае с conll2003 существуют лэйблы следующих видов:

* 'PER' для имен и фамилий
* 'ORG' для названия организаций
* 'LOC' для локаций
* 'MISC' для смешанных сущностей
* 'O' для обычных слов

Также вначале лэйблов бывают буквы B и I. B означает начало сущности, I необходимо для следующего слова, означающего эту же сущность.

In [ ]:
label_list = datasets["train"].features[f"ner_tags"].feature.names
label_list

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']

Посмотрим на пример из датасета:

In [ ]:
example = datasets["train"][4]
print(example.keys())
print(example['tokens'])
print(example['ner_tags'])

dict_keys(['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'])
['Germany', "'s", 'representative', 'to', 'the', 'European', 'Union', "'s", 'veterinary', 'committee', 'Werner', 'Zwingmann', 'said', 'on', 'Wednesday', 'consumers', 'should', 'buy', 'sheepmeat', 'from', 'countries', 'other', 'than', 'Britain', 'until', 'the', 'scientific', 'advice', 'was', 'clearer', '.']
[5, 0, 0, 0, 0, 3, 4, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0]


Для каждого отдельного слова есть номер соответствующего лэйбла.

Загрузим токенизатор:

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

Вспомним, что модели семейства BERT используют subword токенизацию, то есть одно слово может получить несколько отдельных токенов.

In [ ]:
tokenized_input = tokenizer(example["tokens"], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
print(tokens)
print("Всего слов:", len(example["tokens"]))

['[CLS]', 'germany', "'", 's', 'representative', 'to', 'the', 'european', 'union', "'", 's', 'veterinary', 'committee', 'werner', 'z', '##wing', '##mann', 'said', 'on', 'wednesday', 'consumers', 'should', 'buy', 'sheep', '##me', '##at', 'from', 'countries', 'other', 'than', 'britain', 'until', 'the', 'scientific', 'advice', 'was', 'clearer', '.', '[SEP]']
Всего слов: 31


Это означает, что нам необходимо конвертировать лэйблы таким образом, чтобы они соответствовали токенам.

Для того, чтобы проверить к какому слову относится тот или иной токен удобно использовать следующую функцию:

In [ ]:
print(tokenized_input.word_ids())

[None, 0, 1, 1, 2, 3, 4, 5, 6, 7, 7, 8, 9, 10, 11, 11, 11, 12, 13, 14, 15, 16, 17, 18, 18, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, None]


В исходном тексте было 31 слово, столько же индексов выдал и метод ```word_ids()```

### Написание функции для преобразования лэйблов (3 балла)

Ваша задача заключается в том, чтобы написать функцию ```tokenize_and_align_labels()```, которая должна делать токенизацию и преобразовывать лэйблы в формат, соответствующий токенам.

То есть:
* Если слово получило отдельный токен, то ему соответствует один лэйбл
* Если слово получило несколько токенов, то ему должно соответствовать столько же лэйблов. Например, слово crisps получает токенизацию [15594, 2015], тогда в лэйблами для него будет [0, 0]
* Если токен является служебным (имеет индекс None при вызове ```word_ids()```), то ему должен соответствовать лэйбл -100. Это специальный индекс, обозначающий те лэйблы, для которых не нужно считать лосс-функцию.

Пример:

Исходные слова: ```['Only', 'France',
 'and',
 'Britain',
 'backed',
 'Fischler',
 "'s",
 'proposal',
 '.']```

 Исходные лэйблы: ```[0, 5, 0, 5, 0, 1, 0, 0, 0]```

 После токенизации: ```[101, 2069, 2605, 1998, 3725, 6153, 27424, 2818, 3917, 1005, 1055, 6378, 1012, 102]```

 Измененные лэйблы: ```[-100, 0, 5, 0, 5, 0, 1, 1, 1, 0, 0, 0, 0, -100]```

 Также дополнительные примеры можно посмотреть в следующих ячейках ноутбука, которые проверяют корректность реализации функции ```tokenize_and_align_labels()```

In [ ]:
label_all_tokens = True

def tokenize_and_align_labels(examples):
    # Токенизируем текст
    tokenized_inputs = tokenizer(
        examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []  # В этот массив будем складывать id лэйблов токенов
    for i, label in enumerate(examples["ner_tags"]):

        word_ids = tokenized_inputs.word_ids(batch_index=i)

        # Напишите код здесь. Соберите в список label_ids лэйблы, соответствующие токенам

        label_ids = []
        cur_word = None

        for word_id in word_ids:

          # Case 1: The start of the new word is met.
          # Случай 1: Встречено начало нового слова.

          if word_id != cur_word:

            cur_word = word_id

            # If a special token is met, set its label to -100.
            # The reason to use -100 is that during the cross-entropy
            # it is the index that is ignored by the loss function.

            # Если был встречен специальный токен, присваивается лэйбл
            # равный -100. Причина использования -100 заключается в том,
            # что во время кросс-энтропии этот индекс игнорируется функцией потерь.

            lbl = -100

            if word_id != None:
              lbl = label[word_id]

            label_ids.append(lbl)

          # Case 2: A special token is met.
          # Случай 2: Встречен специальный токен.

          elif word_id == None:

            label_ids.append(-100)

          # Case 3: The part of the previous word is met.
          # Случай 3: Встречена часть предыдущего слова.

          else:

            # In the case if the label still means that it is "beginning
            # of the word", change it so that it means "inside the word".

            # В случае если лэйбл всё ещё означет "начало слова", поменять
            # его, чтобы он начал означать "внутри слова".

            # B-... -> I-...

            lbl = label[word_id] + 1 if label[word_id] % 2 else label[word_id]
            label_ids.append(lbl)

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs


In [ ]:
test_examples = {
    'id': ['0', '1', '2'],
    'tokens': [
        ['EU', 'rejects', 'German', 'call', 'to', 'boycott', 'British', 'lamb', '.'],
        ['Peter', 'Blackburn'],
        ['BRUSSELS', '1996-08-22']
        ],
    'ner_tags': [
        [3, 0, 7, 0, 0, 0, 7, 0, 0],
        [1, 2],
        [5, 0]
        ]
    }

test_outputs = {
    'input_ids': [
        [101, 7327, 19164, 2446, 2655, 2000, 17757, 2329, 12559, 1012, 102],
        [101, 2848, 13934, 102],
        [101, 9371, 2727, 1011, 5511, 1011, 2570, 102]
        ],
    'attention_mask': [
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1]
        ],
    'labels': [
        [-100, 3, 0, 7, 0, 0, 0, 7, 0, 0, -100],
        [-100, 1, 2, -100],
        [-100, 5, 0, 0, 0, 0, 0, -100]
        ]
    }

In [ ]:
assert test_outputs == tokenize_and_align_labels(test_examples), "Похоже tokenize_and_align_labels работает не так, как должна"

Применим функцию ко всем выборкам датасета с помощью метода ```map()```

In [ ]:
tokenized_datasets = datasets.map(tokenize_and_align_labels, batched=True)

## Тренировака модели


Обучать модель будем с помощью ```Trainer``` из библиотеки ```transformers```.

Загрузим претренированные веса:

In [ ]:
model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list))

Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Определение аргументов для тренировки (1 балл)

Загляните в [документацию](https://huggingface.co/docs/transformers/main_classes/trainer#transformers.TrainingArguments) и заполните необходимые аргументы для тренировки. Помните, что для файнтюнинга больших моделей следует выбирать небольшой learning rate (обычно меньше 1е-5).

In [ ]:
# This cell is needed in order for the next one
# to work properly. If it still doesn't, restart
# the runtime and try again.

# Эта ячейка необходима для того, чтобы следующая
# работа правильно. Если запустить всё равно не получается,
# попробуйте перезагрузить runtime и попробовать снова.

!pip install transformers[torch]
!pip install accelerate -U

In [ ]:
model_name = model_checkpoint.split("/")[-1]

args = TrainingArguments(
    # Опишите здесь необходимые аргументы

    # The "evaluation_strategy" parameter is needed in order to
    # identify when the evaluation is done. It is set to "epoch"
    # which means that evaluation happens at the end of each epoch.

    # Параметр "evaluation_strategy" необходим для определения того,
    # когда происходит evaluation. Выбрано значение "epoch", которое означает,
    # что evaluation происходит в конце каждой эпохи.

    evaluation_strategy = "epoch",

    # The "num_train_epochs" parameter is needed in order to
    # set the number of epochs during which the model will train.

    # Параметр "num_train_epochs" необходим для того, чтобы задать
    # количество эпох, во время которых модель будет обучаться.

    num_train_epochs = 6,

    # The "learning_rate" parameter is needed in order to
    # set the initial learning rate.

    # Параметр "learning_rate" необходим для того, чтобы
    # задать изначальное значение коэффициента скорости обучения.

    learning_rate = 3e-6,

    # The parameter "output_dir" is needed in order to
    # identify the output directory where the model
    # predictions and checkpoints will be written.

    # Параметр "output_dir" необходим для того, чтобы обозначить
    # вызодной каталог, где будут описаны предсказания и контрольные
    # точки модели.

    output_dir = "./results"
)

Создадим вспомогательные объекты: ```DataCollatorForTokenClassification``` и ```metric```

In [ ]:
data_collator = DataCollatorForTokenClassification(tokenizer)
metric = load_metric("seqeval")

<ipython-input-19-053592a2c97e>:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("seqeval")


### Расчет метрики (1 балл)

Опишем функцию ```compute_metric```, которая будет учитывать только нужные токены.

In [ ]:
import numpy as np

# Changed the passing arguments for compute_metrics().
# When it was run in the first place, the error showed
# that the funtion did not receive the second parameter.
# In order to fix it, one parameter is expected, which is
# then separated into two different.

# Входные параметры для compute_metrics() были изменены.
# Когда программа была запущена в первый раз, ошибка показала,
# что функция не получила второй параметр. Чтобы исправить эту
# ошибку, был сделан один ожидаемый параметр, который впоследствии
# разбивается на два различных.

def compute_metrics(ep_predictions):
    prds, labels = ep_predictions
    predictions = np.argmax(prds, axis=2)

    # Удалим из подсчета метрик служебные токены
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    # По аналогии с фильтрацией true_predictions опишите фильтрацию для true_labels

    # In order to perform the filtration, the following must be done:
    # 1. Remove all the special tokens that were initialized by -100.
    # 2. Convert all the integers to the string format (labels).

    # Чтобы произвести фильтрацию, необходимо выполнить следующее:
    # 1. Убрать все специальные токены, которым были присвоены значения -100.
    # 2. Конвертировать все числа в формат строки (лэйблы).

    true_labels =  [[label_list[lbl] for lbl in label if lbl != -100] for label in labels] # магия здесь

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

### Использование ```Trainer``` для обучения (2 балла)

Далее создайте объект класса ```Trainer``` с необходимыми аргументами и обучите модель.

Подробнее о том, как использовать ```Trainer```, можно почитать [здесь](https://huggingface.co/docs/transformers/main_classes/trainer) или же посмотреть семинарское занятия из этого модуля.  

In [ ]:
# Создайте объект класса Trainer и обучите модель

trainer = Trainer(

    # The "model" parameter is needed in order to
    # declare the model which will be trained,
    # evaluated or used for predicitions.

    # Параметр "model" необходим для того, чтобы обозначить
    # модель, которую будут обучать, оценивать или
    # использовать для предсказаний.

    model = model,

    # The "args" parameter is needed in order to
    # set the set of arguments which will be tweaked
    # during the process of training.

    # Параметр "args" необходим для того, чтобы установить
    # набор аргументов, которые будут участвовать в процессе
    # обучения.

    args = args,

    # The "train_dataset" parameter is needed in order to
    # define the dataset which will be used for training.

    # Пармаетр "train_dataset" необходим для того, чтобы
    # обозначить набор данных, на котором модель будет обучаться.

    train_dataset = tokenized_datasets["train"],

    # The "eval_dataset" parameter is needed in order to
    # define the dataset which will be used for evaluation.

    # Пармаетр "eval_dataset" необходим для того, чтобы
    # обозначить набор данных, на котором модель будет оцениваться.

    eval_dataset = tokenized_datasets["validation"],

    # The "data_collator" parameter is needed in order to
    # declare the function which will form a batch from a list
    # of elements of train_dataset or eval_dataset.

    # Параметр "data_collator" необходим для того, чтобы обозначить
    # функцию, которая сформирует батч из набора элементов из train_dataset
    # и eval_dataset.

    data_collator = data_collator,

    # The "compute_metrics" parameter is needed in order to
    # declare the function that will be used to compute metrics at evaluation.

    # Параметр "compute_metrics" необходим для того, чтобы обозначить
    # функцию, которая будет использована в расчётах метрик при оценивании.

    compute_metrics = compute_metrics,

    # The "tokenizer" parameter is needed in order to
    # identify the operation of how the data will be preprocessed.

    # Параметр "tokenizer" необходим для того, чтобы обозначить операцию
    # того, как данные будут предобработаны.

    tokenizer = tokenizer
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: ignored

### Получение необходимой метрики (3 балла)

 Хорошее качество для этой задачи ~0.92 по F1 мере или выше. Попробуйте добиться этого значения, используя различные гиперпараметры в ```TrainingArguments```. Напишите вывод о проделанной работе.

In [ ]:
# Код

# As it can be seen from the information above, the number of epochs
# is suitable (as the F1-metric and accuracy don't fall, there is no
# overtraining). Let's try to modify the "learning_rate" and see what happens.

# Как можно увидеть из информации выше, количество эпох подобрано подобающе
# (метрика F1 и точность не падают, переобучения не происходит). Попробуем
# изменить параметр "learning_rate" и посмотреть, что получится.

args_mod = TrainingArguments(
    evaluation_strategy = "epoch",
    num_train_epochs = 6,
    learning_rate = 1e-5,
    output_dir = "./results"
)

trainer_mod = Trainer(
    model = model,
    args = args_mod,
    train_dataset = tokenized_datasets["train"],
    eval_dataset = tokenized_datasets["validation"],
    data_collator = data_collator,
    compute_metrics = compute_metrics,
    tokenizer = tokenizer
)

trainer_mod.train()

In [ ]:
# Now it can be seen that F1-metric has improved. Nevertheless, the accuracy
# has fallen on the sixth epoch. There is an overtraining, so the number of
# epochs must be decreased (for these parameters it will be decreased by one).

# Теперь можно увидеть, что метрика F1 улучшилась. Тем не менее точность упала
# на шестой эпохе. Происходит переобучение, поэтому количество эпох должно
# быть уменьшено (для данных параметров оно будет уменьшено на одно).

args_fin = TrainingArguments(
    evaluation_strategy = "epoch",
    num_train_epochs = 5,
    learning_rate = 1e-5,
    output_dir = "./results"
)

trainer_fin = Trainer(
    model = model,
    args = args_fin,
    train_dataset = tokenized_datasets["train"],
    eval_dataset = tokenized_datasets["validation"],
    data_collator = data_collator,
    compute_metrics = compute_metrics,
    tokenizer = tokenizer
)

trainer_fin.train()

### At the end it can be concluded that even though throught the training metrics fluctuated, the overall result has improved.

### В конечном итоге можно сделать вывод, что несмотря на то что в процессе обучения показатели метрик колебались, конечный результат улучшился.

# Summary

# Вывод

### This work has helped me to understand better what is the "Token classification" task about and gave me a hands on experience with NER, how different aspects for this problem are tuned. Steps which were implemented during the work on this notebook were:

* Studying the problem
* Studying the given dataset
* Data preparation and procession
* Metric calculation
* Adjusting needed parameters and tuning the model
* Training the model

### Even though the first metrics that were obtained with the initial "args" are not qualified as "good quality", the research on how to improve them has also helped to understand that the overall performance of the model depends heavily not only on the programming skills of the developer but also on the ability to understand and analyse the task from the mathematical point of view.

### Эта работа позволила мне лучше понять в чём заключается задача классификации токенов и дала мне опыт работы с NER, каким образом настраиваются разные аспекты этой задачи. Шаги, которые были выполнены в процессе работы, включают в себя:

* Изучение поставленной проблемы
* Изучение набора данных
* Подготовка и обработка данных
* Подсчёт метрики
* Выбор нужных параметров и настройка модели
* Тренировка модели

### Несмотря на то что первые полученные метрики с изначальными параметрами "args" не квалифицируются как хорошее качество, исследование о том, как их исправить, также позволило осознать, что общие показатели модели сильно зависят не только от навыков программирования разработчика, но также от способности понять и проанализировать задачу с математическо точки зрения.

### Дополнительный эксперимент (2 балла)

А теперь попробуйте решить ту же задачу, но с другой претренированной моделью из семейства BERT, например, ```roberta-base``` или ```distillroberta-base``` и получить качество выше 0.94 по F1 на валидационном датасете. Список доступных моделей можно посмотреть [здесь](https://huggingface.co/models). Вы на практике убедитесь, насколько различные претренированные модели могут улучшать конечное качество на downstream задачах.

Для выполнения этого пункта можно всего лишь скопировать некоторые ячейки кода выше и поменять переменную ```model_checkpoint``` на название другой модели.

In [ ]:
# Проведите эксперимент здесь

# The following realisation is done for "roberta-base" model.
# Because of the limited resources, the following example shows
# the performance around 0,948 on F1-metric. The parameters can be adjusted,
# so that above 0,95 is reached. I tried 5 epochs and 1e-5 learning rate, which
# resulted in above 0,95 score on F1-metric. Nevertheless, the overtraining was
# discovered. To sum up, the parameters given in this example help to obtain
# the score above 0,94, however the result can be improved for which
# additional resources are required for Google Colab.
# (repeated training for checking different parameters)

# Следующая реализация выполнена для модели "roberta-base".
# Из-за ограниченных ресурсов, приведенный пример показывает около 0,948
# по шкале F1. Параметры могут быть подобраны таким образом, что показатели
# выше 0,95 достигаются. Я пробовал 5 эпох и коэффициент скорости обучения
# равный 1e-5, которые помогли получить результат выше 0,95 по шкале F1.
# Тем не менее, было обнаружено переобучение. В итоге данные параметры
# помогают достичь показатель выше 0,94, однако результат может быть
# улучшен для чего понадобится больше ресурсов Google Colab.
# (повторное обучение для проверки различных параметров)

model_exp = "roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_exp, add_prefix_space=True)
tokenized_datasets_experiment = datasets.map(tokenize_and_align_labels, batched=True)
model_experiment = AutoModelForTokenClassification.from_pretrained(model_exp, num_labels=len(label_list))
data_collator = DataCollatorForTokenClassification(tokenizer)

args_experiment = TrainingArguments(
    evaluation_strategy = "epoch",
    num_train_epochs = 4,
    learning_rate = 5e-6,
    output_dir = "./results"
)

trainer_experiment = Trainer(
    model = model_experiment,
    args = args_experiment,
    train_dataset = tokenized_datasets_experiment["train"],
    eval_dataset = tokenized_datasets_experiment["validation"],
    data_collator = data_collator,
    compute_metrics = compute_metrics,
    tokenizer = tokenizer
)

trainer_experiment.train()